In [5]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt



In [2]:
# Create a Spark session
spark = SparkSession.builder.appName("VectorAssemblerExample").getOrCreate()

23/12/18 18:31:24 WARN Utils: Your hostname, alvaro-ThinkPad-X1-Carbon-Gen-10 resolves to a loopback address: 127.0.1.1; using 10.156.4.72 instead (on interface wlp0s20f3)
23/12/18 18:31:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/18 18:31:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:


# Sample data
data = [(1.60193653, -1.8679101),
        (-1.1328963, -1.9607465),
        (2.40675869, -1.5994823),
        (0.09330145, -2.9446696),
        (1.3795901, -2.5489864),
        (-0.42065496, -2.8165693),
        (0.55753398, -2.0145494),
        (1.3066549, -2.3208153),
        (0.66224722, -0.9406476),
        (1.19072851, -2.4178092),
        (4.67961769, 1.6375689),
        (5.03015133, 1.4575724),
        (6.1003413, 2.1673923),
        (4.20259176, 1.8237144),
        (4.93339445, 1.8983999),
        (6.70975052, 1.5899655),
        (5.01461979, 1.6051478),
        (5.00005277, 1.6855351),
        (4.12926186, 2.0582398)]

columns = ["x1", "x2"]

df2 = spark.createDataFrame(data, columns)

df2.show()


+-----------+----------+
|         x1|        x2|
+-----------+----------+
| 1.60193653|-1.8679101|
| -1.1328963|-1.9607465|
| 2.40675869|-1.5994823|
| 0.09330145|-2.9446696|
|  1.3795901|-2.5489864|
|-0.42065496|-2.8165693|
| 0.55753398|-2.0145494|
|  1.3066549|-2.3208153|
| 0.66224722|-0.9406476|
| 1.19072851|-2.4178092|
| 4.67961769| 1.6375689|
| 5.03015133| 1.4575724|
|  6.1003413| 2.1673923|
| 4.20259176| 1.8237144|
| 4.93339445| 1.8983999|
| 6.70975052| 1.5899655|
| 5.01461979| 1.6051478|
| 5.00005277| 1.6855351|
| 4.12926186| 2.0582398|
+-----------+----------+



In [7]:
# Create a VectorAssembler
assembler = VectorAssembler(inputCols=["x1", "x2"], outputCol="features")
dataset = assembler.transform(df2)

dataset.show()

+-----------+----------+--------------------+
|         x1|        x2|            features|
+-----------+----------+--------------------+
| 1.60193653|-1.8679101|[1.60193653,-1.86...|
| -1.1328963|-1.9607465|[-1.1328963,-1.96...|
| 2.40675869|-1.5994823|[2.40675869,-1.59...|
| 0.09330145|-2.9446696|[0.09330145,-2.94...|
|  1.3795901|-2.5489864|[1.3795901,-2.548...|
|-0.42065496|-2.8165693|[-0.42065496,-2.8...|
| 0.55753398|-2.0145494|[0.55753398,-2.01...|
|  1.3066549|-2.3208153|[1.3066549,-2.320...|
| 0.66224722|-0.9406476|[0.66224722,-0.94...|
| 1.19072851|-2.4178092|[1.19072851,-2.41...|
| 4.67961769| 1.6375689|[4.67961769,1.637...|
| 5.03015133| 1.4575724|[5.03015133,1.457...|
|  6.1003413| 2.1673923|[6.1003413,2.1673...|
| 4.20259176| 1.8237144|[4.20259176,1.823...|
| 4.93339445| 1.8983999|[4.93339445,1.898...|
| 6.70975052| 1.5899655|[6.70975052,1.589...|
| 5.01461979| 1.6051478|[5.01461979,1.605...|
| 5.00005277| 1.6855351|[5.00005277,1.685...|
| 4.12926186| 2.0582398|[4.1292618

In [8]:
# Create K-Means model
kmeans = KMeans(featuresCol="features", k=2)
model = kmeans.fit(dataset)

# Print cluster centers
print("Cluster Centers: ")
for center in model.clusterCenters():
    print(center)


##### USE THE MODEL EXAMPLE

Cluster Centers: 
[5.08886461 1.76928179]
[ 0.76452001 -2.14321857]


In [10]:
result = dataset.select("features", "prdiction").toPandas()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `prdiction` cannot be resolved. Did you mean one of the following? [`x1`, `x2`, `features`].;
'Project [features#128, 'prdiction]
+- Project [x1#0, x2#1, UDF(struct(x1, x1#0, x2, x2#1)) AS features#128]
   +- LogicalRDD [x1#0, x2#1], false


In [ ]:
# Visualize the data
plt.scatter(result["feature1"], result["feature2"], c=result["prediction"], cmap="viridis")
plt.scatter(model.clusterCenters()[:, 0], model.clusterCenters()[:, 1], marker="x", s=200, linewidths=3, color="r")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.title("KMeans Clustering Results")
plt.show()

In [ ]:
# Stop the Spark session
spark.stop()